In [1]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role
import time
import pandas as pd
import matplotlib.pyplot as plt
from helper import *


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

s3_bucket = sagemaker_session.default_bucket()
s3_prefix = 'MLEND-Capstone-Project'    

region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [3]:
stock_data_preprocessed = pd.read_csv('stock_indicator_data.csv',parse_dates=True, index_col=[0,1])
stock_data_preprocessed = stock_data_preprocessed[['Adj Close','Volume','target']]
get_target_distribution(stock_data_preprocessed)


-1:  363122
 0:  357809
 1:  456392


In [4]:
stock_data_preprocessed

Adj Close    Volume  target
Date       Ticker                             
2010-03-16 A        0.005604  0.003822       1
           AAL      0.001880  0.006881      -1
           AAP      0.010572  0.000981       1
           AAPL     0.007148  0.129969       0
           ABC      0.006258  0.004898       1
...                      ...       ...     ...
2019-12-31 YUM      0.025759  0.001479       0
           ZBH      0.038353  0.000712       0
           ZBRA     0.065616  0.000423       0
           ZION     0.013238  0.001450       0
           ZTS      0.033893  0.001313       0

[1177323 rows x 3 columns]

In [5]:
tickers = get_sp500_tickers()

freq = 'D'

# we predict for 1 day
prediction_length = 1

# we use 50 days as context length, this is the number of state updates accomplished before making predictions
context_length = 50

end_training = pd.Timestamp('2018-12-31', freq=freq)

timeseries = []
    
for ID,ticker in list(enumerate(tickers)):
    ticker = stock_data_preprocessed.loc[(slice(None), ticker), :]
    if ticker.index[0][0]<end_training:
        timeseries.append(ticker)

In [6]:
training_data = [
    {
            "start": str(ts.index[0][0]),
            "target": ts['target'][ts.index[0][0]:end_training].tolist(), # We use -1, because pandas indexing includes the upper bound 
            "dynamic_feat": ts[['Adj Close','Volume']][ts.index[0][0]:end_training].values.T.tolist()
    }
    for ts in timeseries
]
print(len(training_data))

491


In [7]:
num_test_windows = 10

test_data = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training + timedelta(days=(2*k * prediction_length))].tolist(),
        "dynamic_feat": ts[['Adj Close','Volume']][ts.index[0][0]:end_training + timedelta(days=(2*k * prediction_length))].values.T.tolist()
    }
    for k in range(1, num_test_windows + 1) 
    for ts in timeseries
]
print(len(test_data))

4910


In [8]:
%%time
write_json_dataset("price_train.json", training_data)
write_json_dataset("price_test.json", test_data)

CPU times: user 18.4 s, sys: 860 ms, total: 19.3 s
Wall time: 22.1 s


In [9]:
%%time
copy_to_s3("price_train.json", s3_data_path + "/train/train.json", s3_bucket)
copy_to_s3("price_test.json", s3_data_path + "/test/test.json", s3_bucket)

File s3://sagemaker-us-east-2-017500148529/s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data/train/train.json already exists.
Set override to upload anyway.

File s3://sagemaker-us-east-2-017500148529/s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data/test/test.json already exists.
Set override to upload anyway.

CPU times: user 146 ms, sys: 3.91 ms, total: 150 ms
Wall time: 234 ms


In [10]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

{"start": "2010-03-16 00:00:00", "target": [1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1...


In [11]:
price_estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='MLEND-Capstone-Project',
    output_path=s3_output_path
)

In [12]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "30",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_dynamic_feat": 'auto',
}

In [13]:
price_estimator.set_hyperparameters(**hyperparameters)

In [14]:
%%time
data_channels = {
    "train": s3_data_path + "/train/train.json",
    "test": s3_data_path + "/test/test.json"
}

price_estimator.fit(inputs=data_channels, wait=True)

2020-06-09 04:10:49 Starting - Starting the training job...
2020-06-09 04:10:51 Starting - Launching requested ML instances......
2020-06-09 04:11:53 Starting - Preparing the instances for training...
2020-06-09 04:12:40 Downloading - Downloading input data...
2020-06-09 04:13:20 Training - Training image download completed. Training in progress..Arguments: train
[06/09/2020 04:13:23 INFO 140436758837056] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u'

In [16]:
#price_predictor = price_estimator.deploy(
#    initial_instance_count=1,
#    instance_type='ml.m4.xlarge',
#    content_type="application/json")

In [22]:
price_predictor = sagemaker.predictor.RealTimePredictor(endpoint='price-endpoint')

In [23]:
dynamic_feat = ['Adj Close','Volume']
date = '2019-01-02'
ticker = 'AAPL'
df = stock_data_preprocessed
predictor = price_predictor
def get_dynamic_feat_prediction(ticker,date,df,predictor,dynamic_feat):

    date_pred = pd.Timestamp(date, freq='D')
    date_start = date_pred-timedelta(days=50)
    pred_df = stock_data_preprocessed.loc[(slice(str(date_start),str(date_pred)), ticker), :]
    result_df = pred_df.loc[(slice(str(date_pred),str(date_pred)), ticker), :]
    pred = {
            "start": str(date_pred),
            "target": pred_df['target'][date_start:date_pred-timedelta(days=1)].tolist(),
            "dynamic_feat": pred_df[dynamic_feat][date_start:date_pred].values.T.tolist()
        }

    req = encode_request(instance=pred, num_samples=50, quantiles=['0.1', '0.5', '0.9'])
    res = price_predictor.predict(req)
    prediction_data = json.loads(res.decode('utf-8'))
    pred = round(prediction_data['predictions'][0]['quantiles']['0.5'][0])
    result_df['prediction'] = pred


    return result_df

get_dynamic_feat_prediction(ticker,date,df,predictor,dynamic_feat)

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,,Adj Close,Volume,target,prediction
Date,Ticker,,,,
2019-01-02,AAPL,0.03987,0.043087,-1,-1


In [30]:
date_index = pd.read_csv('test_date_index.csv')
date_index = date_index.values.reshape(252).tolist()

def get_dynamic_feat_accuracy(ticker):
    i = 0
    target = []
    prediction = []
    df = stock_data_preprocessed
    for date in date_index:
        target.append(get_dynamic_feat_prediction(ticker, date,df,price_predictor,dynamic_feat)['target'].values[0])
        prediction.append(int(get_dynamic_feat_prediction(ticker, date,df,price_predictor,dynamic_feat)['prediction'].values[0]))
    target = list(np.array(target).reshape(252))
    prediction = list(np.array(prediction).reshape(252))
    data = {'target': list(target), 'prediction': list(prediction)}
    prediction_df = pd.DataFrame(data=data,index=date_index, columns=['target','prediction'])

    return accuracy_score(target, prediction)

In [31]:
get_dynamic_feat_accuracy(ticker='F')

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.7063492063492064

In [32]:
acc = {}

In [ ]:
acc['A'] = get_dynamic_feat_accuracy('A')
acc['F'] = get_dynamic_feat_accuracy('F')
acc['GE'] = get_dynamic_feat_accuracy('GE')
acc['DAL'] = get_dynamic_feat_accuracy('DAL')
acc['UAL'] = get_dynamic_feat_accuracy('UAL')
acc['ABC'] = get_dynamic_feat_accuracy('ABC')
acc['CAT'] = get_dynamic_feat_accuracy('CAT')
acc['DE'] = get_dynamic_feat_accuracy('DE')
acc['D'] = get_dynamic_feat_accuracy('D')


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
np.array(list(acc.values())).mean()